앞서 \ref{chap:TyInf}장에서 TODO
\lipsum[3-3]

TODO 일단 12장 내용을 템플릿으로 복사만 해 놓은 상태 TODO

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

\newpage
# FACT언어에서 타입 표기의 생략 허용
앞서 \ref{chap:TyChk}장에서 살펴본 FACT언어의 문법구조(\ref{fig:FACTlang})에서는
함수요약식과 재귀함수요약식에서 유효범휘에 묶이는 이름의 타입이 무엇인지 항상 나타난다.
우선, 함수요약식과 재귀함수요약식에서 타입 정보가 생략된 경우를 허용하도록,
즉 $\lambda x.e$나 $\textbf{rec}\,f\,(\lambda\cdots)$와 같은 형태도 허용하도록,
지난 \ref{chap:TyChk}장에서 FACT언어의 문법구조를 표현하던 `Expr`을
아래와 같이 약간 수정하였다. 수정된 부분은 `Lam`과 `Rec`의
선언 부분에서 기존의 `Ty` 대신 나타나는 `Maybe Ty`다.
(새로 추가된 $\textbf{let}$식은 \ref{sec:LetSugar}절에서 다룰 예정이며, 일단은 없는 것으로 생각하자.)

In [2]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm                      -- x
          | Lam Nm (Maybe Ty) Expr      -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr      -- rec f:t (\x:t2.e)
          | App Expr Expr               -- (e1 e2)
          | Lit Int                     -- n
          | Add Expr Expr               -- e1 + e2
          | If Expr Expr Expr           -- if e then e1 else e0
          | Let Nm (Maybe Ty) Expr Expr -- let x:t = e2 in e
          deriving (Eq, Ord)
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0
  showsPrec p (Let x mt e2 e) = showParen (p > 2) $
      showString("\\textbf{let}\\,"++x) . showsMT mt .
      showString " = " . showsPrec 3 e .
      showString "\\,\\textbf{in}\\," . showsPrec 3 e2

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Ty where
    display t = display [latex $ "$"++show t++"$"]

In [4]:
idExpr = Lam "x" Nothing (Var "x")
ttExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "x")) 
ffExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "y")) 
-- 일부 이름 바인딩에 타입 표기가 생략된 예시 프로그램
If (Add (Lit 3) (Var "y"))  ffExpr
                            (App idExpr (Rec "f" Nothing ffExpr))

In [5]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay TEnv where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Val where
    display v = display [latex $ "$"++show v++"$"]

타입검사 함수 `check`와 타입유추 함수 `infer`는 기존의 내용을 사실상 그대로 재사용할 것이다.
타입 표기가 전혀 생략되지 않은 식, 즉 `Nothing`이 하나도 나타나지 않는 식에 대해서만
`check`의 결과가 참(`True`)이 될 수 있으며 `infer`의 결과도 비어있지 않고 유추된 타입이
들어있는 리스트(`[`$\tau\!\!$ `]`)가 될 수 있다.

In [6]:
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x (Just t2) e) = undefined
infer tenv (Lam x Nothing   e) = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (Rec f (Just t) e)  = undefined
infer tenv (Rec f Nothing  e)  = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (App e1 e2)  = undefined
infer tevv (Lit _)      = [Z]
infer tenv (Add e1 e2)  = [Z | check tenv e1 Z, check tenv e2 Z]
infer tenv (If e e1 e0) = undefined

\noindent
\ref{chap:TyChk}장에서와 마찬가지 내용으로 위의 `infer`와 아래의 `ev` 함수에서
`undefined`로 처리된 부분을 작성하면 된다.

인터프리터 함수 `ev` 및 `eval'`도 기존의 내용을 그대로 재사용한다.
아래의 인터프리터 함수 `ev`는 타입유추가 성공한 경우, 즉 `infer`의
결과가 빈 리스트가 아닌 경우에만 실행한다고 가정한다.

In [7]:
ev :: Env -> Expr -> Val  -- infer가 성공한 경우에만 실행함을 가정
ev env (Var x) = v    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- 적절히 작성
ev env (App e1 e2) = ev ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = ev env e1 
          v2                  = ev env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)  where Vi n1 = ev env e1
                                         Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1 else ev env e0
    where Vi n = ev env e

In [8]:
-- infer에 성공한 닫힌식에 대한 인터프리터
eval' e = [ev [] e | t <- infer [] e]

In [9]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i t hen i + f(i + -1) else i)

\noindent
0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수 `e4`를 위와 같이 작성하여
다음과 같이 실행해 보자.
`infer`와 `ev`함수의 `undefined` 부분을 잘 채워넣었다면
아래에 보이는 에러 없이 정상적으로 0부터 100까지의 합 5050이 계산될 것이다.

In [10]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval' (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

# 함수요약식 파라메터의 생략된 타입 표기 복원
타입 표기가 생략된 함수요약식이 $(\lambda x.e)~e_2$와 같이 함수적용식의 함수 위치에 있을 때
인자식 $e_2$의 타입유추가 가능하면 생략된 타입 표기를 복원할 수 있다.
즉, $e_2:\tau_2$로 유추된다면 $(\lambda x{:}\tau_2.e)~e_2$로 복원할 수 있다는 말이다.
조금 더 일반적으로는, 연달아 겹쳐진 람다요약식이
와 같이 연쇄적인 함수적용식의 첫머리에 있을 때,
각 인자식의 타입이 $e_{21}:\tau_{21}$, $e_{22}:\tau_{22}$, $\cdots$, $e_{2k}:\tau_{2k}$로 유추된다면,
$(\lambda x_1{:}\tau_{21}.\lambda x_{22}{:}\tau_{22}.\ldots\lambda x_{2k}{:}\tau_{2k}.e)
 ~e_{21}~e_{22}\,\cdots\,e_{2k}$로 각 파라메터의 타입 표기를 복원할 수 있다.
이러한 생략된 타입 표기의 복원 과정을 \`타입복원\`이라 부르자.

이 장에서 취하는 전략은 타입복원을 통해 타입유추의 기능을 확장하려 하는 것이다.
타입복원(`recon`)으로 생략된 타입 표기를 모두 복원할 수 있다면
기존의 타입유추(`infer`)를 적용하면 된다. 가장 단순하게 전체 식이
$(\lambda x.e)~e_2$의 형태인 경우에만 타입을 복원하는 함수 `recon`은
아래처럼 작성할 수 있다. 이 함수의 타입복원 기능을 개선하는 것이 
이번 장의 연습문제들로 구성되어 있다.

In [11]:
recon :: TEnv -> Expr -> [Expr]  -- 복원하지 못하면 빈 리스트
recon tenv (App (Lam x Nothing e) e2) =
        [ App (Lam x (Just t2) e) e2 | t2 <- infer tenv e2 ]
recon _    _ = []

In [12]:
e1 = Lam "x" Nothing . Lam "x'" Nothing $ Var "x" `Add` Var "x'"  
e2 = Lit 2 `Add` Var "y"  -- y는 자유변수!!!
App e1 e2

In [13]:
recon [("y",Z)] (App e1 e2) -- 첫 파라메터 x의 타입 표기 복원

\noindent
가장 단순하게 작성한 `recon` 함수로는 위와 같이 첫 파라메터 $x$의 타입 표기는 복원된다.
하지만 아래와 같이 추가로 파라메터를 하나 더 제공하여 연쇄적인 함수적용식을 구성하면 타입 복원에 실패한다.

In [14]:
e2' = Lit 3
App (App e1 e2) e2'

In [15]:
recon [("y",Z)] (App (App e1 e2) e2') -- 복원실패 (빈 리스트 display X)

\noindent
함수 `recon`으로 `App e1 e2`에 대해서는 타입복원에 성공했는데
`App (App e1 e2) e2'`에 대해서는 `x'`는 물론이고 `x`의 타입 표기 복원조차 못하고 실패하는 이유를 생각해 보고,
연습문제에 지시된 내용에 따라 `recon` 함수를 개선해 나가 보라.

다음 예시처럼 함수 인자 부분인 `e5`에도 복원가능한 생략된 타입 표기가 있을 경우까지 고려하여,
`e5`를 타입유추(`infer`)가 가능한 형태로 타입복원하여 전체 `App e1 e5`의 타입복원을 
진행하는 기능까지 포함하도록 `recon` 함수를 개선하는 것도 연습문제이다.

In [16]:
e5 = Lam "z" Nothing (Var "z") `App` Lit 5
App e1 e5

In [17]:
-- 현재 infer에서 undefined로 남겨진 부분에서 에러가 나는데 infer를
recon [("y",Z)] (App e1 e5) -- 제대로 작성해도 빈 리스트로 실패함

: 

In [18]:
recon [("y",Z)] e5 -- e5는 현재 recon 함수로도 타입복원 가능

In [19]:
-- 현재 infer에서 undefiend로 남겨진 부분에서 에러가 나는데 
-- infer를 제대로 작성하면 x에 대한 타입 표기 복원에 성공할 것임
[ recon [("y",Z)] (App e1 e5') | e5' <- recon [("y",Z)] e5 ] 

: 

# 재귀함수요약식에 대한 타입복원
지금 이 장에서 다루는 타입복원의 방식으로는, $\textbf{rec}\,f\,(\lambda x:\tau_2.e)$와 같이
재귀함수 자신을 나타내는 $f$의 생략된 타입 표기를 일반적으로 복원하기 어렵다.
함수 파라메터 $x$의 타입 표기가 생략되지 않은 경우라 할지라도 말이다.
이렇게 $f$의 타입 표기가 생략된 경우에까지 처리할 수 있는 타입유추 방식에 대해서는
이후 \ref{chap:tyInfMore}장에서 다룬다.

따라서, $\textbf{rec}\,f{:}\tau_2\to\tau\,(\lambda x.e)$처럼 재귀함수 자신을 나타내는 $f$의 타입 표기가
생략되지 않은 경우에만 생략된 함수 파라메터 $x$의 타입을 $\tau_2$로 복원하여
$\textbf{rec}\,f{:}\tau_2\to\tau\,(\lambda x{:}\tau_2.e)$로 결과를 계산하는
간단한 타입복원 기능만을 이번 장에서는 구현하도록 하자.

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. C++과 Java는 모든 지역변수의 타입을 반드시 표기해야 하는 언어로 시작했지만
   최근에는 지역변수를 선언할 때 타입을 프로그래머가 구체적으로 표기하지 않아도 컴파일러가
   추측해 주는, 이 장에서 설명한 타입복원과` 같은 기능을 지원하고 있다.
   이러한 기능이 C++이나 Java에서 어떻게 발전했는지 조사해 보라.
1. TODO aaa TODO